In [6]:
import xml.etree.ElementTree as ET
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
import re
import numpy as np

In [7]:
# Parametros para funciones generadoras
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
TEMPLATE_DEVSML = 'template-devsml.xml'

def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)

In [16]:
# xmile2devsml
def xmile2devsml(archivoXmile, archivoDevsml):
    tree = ET.parse(archivoXmile)
    root = tree.getroot()

    model = root.find('model')
    variables = model.find('variables')

    flows  = variables.findall('flow')
    auxs   = variables.findall('aux')
    stocks = variables.findall('stock')
    
    # Flows
    flows_links = {}
    for flow in flows:
        function = flow.find('eqn').text.replace('"', '')
        params_function = filter(lambda x : x != '', re.split("[()-/*]+", function))
        params_function_lower = map(lambda x : x[0].lower() + x[1:].replace(' ', ''), params_function)
        flows_links[flow.get('name')] = {'orig' : None, 
                                         'dest' : None, 
                                         'function' : function,
                                         'params_function' : params_function,
                                         'params_function_lower' : params_function_lower} 
    # Stocks
    stocks_feed = {}
    stockNames  = []
    stock_values = {}
    for stock in stocks:
        stockName = stock.get('name')
        stocks_feed[stockName] = {'fm' : [], 'fp' : []}
        if stock.find('outflow') is not None:
            flowName_out = stock.find('outflow').text.replace('"', '')
            flows_links[flowName_out]['orig'] = stockName
            stocks_feed[stockName]['fm'].append(flowName_out)
        if stock.find('inflow') is not None:
            flowName_in = stock.find('inflow').text.replace('"', '')
            flows_links[flowName_in]['dest'] = stockName
            stocks_feed[stockName]['fp'].append(flowName_in)

        stockNames.append(stockName)
        stock_values[stockName] = stock.find('eqn').text

    # En base a los flows, actualizo 'function' en flow_links
    for linkName, attr in flows_links.iteritems():
        for stockName in stockNames:
            if stockName in attr['function']:
                attr['function'] = attr['function'].replace(stockName, stockName + ' Integrator')
        
    # Aux's
    aux_values = {}
    for aux in auxs:
        aux_values[aux.get('name')] = aux.find('eqn').text

    # Normalizacion de stock_values segun los valores de los aux's
    for stockName, val in stock_values.iteritems():
        if not val.isdigit():
            stock_values[stockName] = aux_values[val]

    #print '*stockNames:',stockNames
    #print '*stock_values:',stock_values
    #print '*flows_links:',flows_links
    #print '*params_function:',params_function
    #print '*params_function_lower:',params_function_lower
    #print '*aux_values:',aux_values
    #print '*stocks_feed:',stocks_feed
    
    context = {
        'stockNames' : stockNames,
        'stock_values' : stock_values,
        'flows_links' : flows_links,
        'params_function' : params_function,
        'params_function_lower' : params_function_lower,
        'aux_values' : aux_values,
        'stocks_feed' : stocks_feed
    }
    
    # Guardar el .devsml en output
    with open(archivoDevsml, 'w') as f:
        f.write(render_template(TEMPLATE_DEVSML,context))

In [18]:
archivoXmile  = 'modelos traducidos/sir.xmile'
archivoDevsml = 'sir/sir-devsml-traduccion.xml'
xmile2devsml(archivoXmile, archivoDevsml)

archivoXmile  = 'modelos traducidos/teacup.xmile'
archivoDevsml = 'teacup/teacup-devsml-traduccion.xml'
xmile2devsml(archivoXmile, archivoDevsml)